In [3]:
#import library
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [4]:
#configuration selenium driver
driver = webdriver.Chrome()

In [5]:
wait = WebDriverWait(driver,10)

In [6]:
#url target
url1 = "https://x.com/i/flow/login"
driver.get(url1)

In [7]:
# input username
username_input = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='text']")))
username_input.send_keys('Your Username')  

next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(),'Next')]")))
next_button.click()

In [8]:
# input password
password_input = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))
password_input.send_keys('Your Password')

login_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(),'Log in')]")))
login_button.click()

In [ ]:
# waiting until load homepage finished
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testid="SideNav_NewTweet_Button"]')))

In [42]:
#target scraping
search_box = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@data-testid='SearchBox_Search_Input']")))
search_box.send_keys("python")
search_box.send_keys(Keys.ENTER)

In [43]:
# Scrolling
for _ in range(8): # Scrolling 8 times
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)

In [44]:
# Extract data
posts = []
post_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[data-testid="cellInnerDiv"]')))
for post_element in post_elements:
    try:
        # extract username
        username_element = post_element.find_element(By.CSS_SELECTOR, '[data-testid="User-Name"] a') 
        username = username_element.text.strip()

        # extract tweet
        text_element = post_element.find_element(By.CSS_SELECTOR, '[data-testid="tweetText"]')
        text = text_element.text.strip()
        
        # extract likes
        likes_element = post_element.find_element(By.CSS_SELECTOR, '[data-testid="like"]')
        likes = likes_element.text.strip()
        
        # extract replies
        replies_element = post_element.find_element(By.CSS_SELECTOR, '[data-testid="reply"]')
        replies = replies_element.text.strip()
        
        # extract retweets
        retweets_element = post_element.find_element(By.CSS_SELECTOR, '[data-testid="retweet"]')
        retweets = retweets_element.text.strip()
        
        posts.append({"username": username, "text": text, "like": likes, "reply": replies, "retweet": retweets})
    except Exception as e:
        print(f"Error extracting data from post: {e}")



In [45]:
# Save to dataframe
df_selenium = pd.DataFrame(posts)
df_selenium.to_csv("posts_selenium.csv", index=False)

In [46]:
driver.quit()

In [47]:
#Show dataframe
df = pd.read_csv("posts_selenium.csv")
df.head()

,username,text,like,reply,retweet
0,Massimo,Jay Brewer collecting eggs from a reticulated ...,375,49,40
1,Monty Python,"Happy birthday to the late, but the ever so gr...",3.2K,146,425
2,Maggie Python,8 lat poniewierki. 8 lat niszczonego nam życia...,3.6K,376,754
3,Bsumettikul,Snake warning \n\nThis is Mochi.\nShe is a fem...,16K,798,5.4K
4,Charlie Holtz,David Attenborough is now narrating my life\n\...,27K,733,6K


In [48]:
df.shape

(7, 5)